In [2]:
from pyspark import SparkContext
from pyspark.mllib.fpm import *
sc = SparkContext("local", "App Name")
from pyspark import SparkConf
conf = (SparkConf()
         .setMaster("local")
         .setAppName("App Name")
         .set("spark.executor.memory", "3g"))
import pandas

In [3]:
#not necessary to include the variable names. with more time, I fix this to leave it out.
names=['class','cap-shape','cap-surf','cap-col','bruise','odor','gill-att',
	'gill-spac','gill-size','gill-color','stalk-shape','stalk-root',
	'stalk-surf-above-ring','stalk-surf-below-ring','stalk-col-above',
	'stalk-col-below','veil-type','veil-col','ring-num','ring-type',
	'spore-col','pop','habitat']
d = pandas.read_csv('agaricus-lepiota.data',header=None, names=names,na_values='?')

In [6]:
#append the column number to each value
for j in range(0,d.shape[1]):
    d[names[j]]=d[names[j]] + str(j)

In [7]:
d['veil-type'].unique()
d[[0]]

KeyError: '[0] not in index'

In [8]:
#all veil-type same, adds no new info, so drop
d=d.drop('veil-type',1)
d2=d.values.tolist()

In [9]:
d2

[['p00',
  'x11',
  's22',
  'n33',
  't44',
  'p55',
  'f66',
  'c77',
  'n88',
  'k99',
  'e1010',
  'e1111',
  's1212',
  's1313',
  'w1414',
  'w1515',
  'w1717',
  'o1818',
  'p1919',
  'k2020',
  's2121',
  'u2222'],
 ['e00',
  'x11',
  's22',
  'y33',
  't44',
  'a55',
  'f66',
  'c77',
  'b88',
  'k99',
  'e1010',
  'c1111',
  's1212',
  's1313',
  'w1414',
  'w1515',
  'w1717',
  'o1818',
  'p1919',
  'n2020',
  'n2121',
  'g2222'],
 ['e00',
  'b11',
  's22',
  'w33',
  't44',
  'l55',
  'f66',
  'c77',
  'b88',
  'n99',
  'e1010',
  'c1111',
  's1212',
  's1313',
  'w1414',
  'w1515',
  'w1717',
  'o1818',
  'p1919',
  'n2020',
  'n2121',
  'm2222'],
 ['p00',
  'x11',
  'y22',
  'w33',
  't44',
  'p55',
  'f66',
  'c77',
  'n88',
  'n99',
  'e1010',
  'e1111',
  's1212',
  's1313',
  'w1414',
  'w1515',
  'w1717',
  'o1818',
  'p1919',
  'k2020',
  's2121',
  'u2222'],
 ['e00',
  'x11',
  's22',
  'g33',
  'f44',
  'n55',
  'f66',
  'w77',
  'b88',
  'k99',
  't1010',
  'e111

In [7]:
rdd=sc.parallelize(d2, 2)

In [138]:
#method of timing https://stackoverflow.com/questions/8220801/how-to-use-timeit-module/29512249#29512249
import timeit
start=timeit.default_timer()
FPGrowth.train(rdd, 0.5, 2).freqItemsets().collect()
print(timeit.default_timer()-start)

In [9]:
len(FPGrowth.train(rdd, 0.5, 2).freqItemsets().collect())

76

In [76]:
patterns50=pandas.DataFrame(FPGrowth.train(rdd, 0.5, 2).freqItemsets().collect())

In [77]:
#get top 12 patterns
patterns50['freq']=patterns50['freq'].astype('int64')
patterns50=patterns50.sort_values('freq',ascending=False)
print(patterns50.reset_index(drop=True)[0:12])

             items  freq
0            [w17]  7924
1             [f6]  7914
2        [f6, w17]  7906
3            [o18]  7488
4        [o18, f6]  7296
5   [o18, f6, w17]  7288
6       [o18, w17]  7288
7             [c7]  6812
8        [c7, w17]  6620
9    [c7, f6, w17]  6602
10        [c7, f6]  6602
11       [c7, o18]  6464


In [ ]:
start=timeit.default_timer()
FPGrowth.train(rdd, 0.1, 2).freqItemsets().collect()
print(timeit.default_timer()-start)

In [ ]:
len(FPGrowth.train(rdd, 0.1, 2).freqItemsets().collect())

In [ ]:
start=timeit.default_timer()
FPGrowth.train(rdd, 0.05, 2).freqItemsets().collect()
print(timeit.default_timer()-start)

In [22]:
len(FPGrowth.train(rdd, 0.05, 2).freqItemsets().collect())

In [22]:
rdd=sc.parallelize(d2, 1000)

In [ ]:
#Memory intensive
#turns out to be 3 minutes, uncomment to run

#start=timeit.default_timer()
#FPGrowth.train(rdd, 0.02,100).freqItemsets().collect() 
#print(timeit.default_timer()-start)

In [23]:
model2perc=FPGrowth.train(rdd, 0.02,100)

In [25]:
#Memory intensive
patterns2perc=model2perc.freqItemsets().collect()

In [26]:
len(patterns2perc)

9612315

In [39]:
import pyfpgrowth
patterns=pyfpgrowth.find_frequent_patterns(d2, 2500)
rules=pyfpgrowth.generate_association_rules(patterns,0.2)

In [107]:
print(*patterns, sep='\n')

{('n8',): 2512, ('n8', 'o18'): 2512, ('f6', 'n8'): 2512, ('n8', 'w17'): 2504, ('f6', 'n8', 'o18'): 2512, ('n8', 'o18', 'w17'): 2504, ('f6', 'n8', 'w17'): 2504, ('f6', 'n8', 'o18', 'w17'): 2504, ('s2',): 2556, ('e19', 'f4'): 2584, ('e19', 'f4', 'w17'): 2576, ('e19', 'f4', 'f6'): 2584, ('e19', 'f4', 'f6', 'w17'): 2576, ('e19', 'o18'): 2584, ('e19', 'f4', 'o18'): 2584, ('e19', 'o18', 'w17'): 2576, ('e19', 'f6', 'o18'): 2584, ('e19', 'f4', 'o18', 'w17'): 2576, ('e19', 'f4', 'f6', 'o18'): 2584, ('e19', 'f6', 'o18', 'w17'): 2576, ('e19', 'f4', 'f6', 'o18', 'w17'): 2576, ('e19', 'w17'): 2768, ('e19', 'f6', 'w17'): 2768, ('e19', 'f6'): 2776, ('c7', 'd22'): 2956, ('c7', 'd22', 'o18'): 2912, ('c7', 'd22', 'f6'): 2938, ('c7', 'd22', 'w17'): 2956, ('c7', 'd22', 'f6', 'o18'): 2912, ('c7', 'd22', 'o18', 'w17'): 2912, ('c7', 'd22', 'f6', 'w17'): 2938, ('c7', 'd22', 'f6', 'o18', 'w17'): 2912, ('d22', 'o18'): 3104, ('d22', 'f6', 'o18'): 3104, ('d22', 'o18', 'w17'): 3104, ('d22', 'f6', 'o18', 'w17'): 31

In [86]:
rules

{('n8',): (('f6', 'o18', 'w17'), 0.9968152866242038),
 ('o18',): (('f6', 'w17'), 0.9732905982905983),
 ('f6',): (('w17',), 0.9989891331817033),
 ('w17',): (('f6',), 0.9977284199899041),
 ('f6', 'n8'): (('o18', 'w17'), 0.9968152866242038),
 ('f6', 'o18'): (('w17',), 0.9989035087719298),
 ('n8', 'o18'): (('f6', 'w17'), 0.9968152866242038),
 ('n8', 'w17'): (('f6', 'o18'), 1.0),
 ('o18', 'w17'): (('f6',), 1.0),
 ('f6', 'w17'): (('o18',), 0.9218315203642803),
 ('f6', 'n8', 'o18'): (('w17',), 0.9968152866242038),
 ('f6', 'n8', 'w17'): (('o18',), 1.0),
 ('f6', 'o18', 'w17'): (('c7',), 0.8605927552140505),
 ('n8', 'o18', 'w17'): (('f6',), 1.0),
 ('e19', 'f4'): (('f6', 'o18', 'w17'), 0.9969040247678018),
 ('e19', 'w17'): (('f6',), 1.0),
 ('f4', 'w17'): (('f6',), 0.996042216358839),
 ('e19', 'f6'): (('w17',), 0.9971181556195965),
 ('f4', 'f6'): (('w17',), 0.998237108858528),
 ('e19', 'f4', 'f6'): (('o18', 'w17'), 0.9969040247678018),
 ('e19', 'f4', 'w17'): (('f6', 'o18'), 1.0),
 ('e19', 'f6', 'w

In [ ]:
##Things that didn't work

#itemstup=tuple()
#for i in range(0,len(patterns50)):
#    itemstup=itesmtup.append(tuple(patterns50["items"][i]))
#patterns50.set_index(patterns50["items"].apply(str)).to_dict()["freq"]
#patterns50.values.tolist()

#patternsforrules=patterns50.to_dict()
#patternsforrules
#pyfpgrowth.generate_association_rules(patternsforrules, 0.001)


#different package to construct itemsets
#from orangecontrib.associate.fpgrowth import *  
#itemsets = frequent_itemsets(d2, .1)
#list(association_rules(itemsets, 0.5))

#attempt with pyfpgrowth
#works for higher threshold, but also crashes at 2%, 163

#import pyfpgrowth
#patterns = pyfpgrowth.find_frequent_patterns(d2, 4062)
#test=pandas.DataFrame([patterns], columns=patterns.keys()).transpose()
#test=test.sort_values(list(test),ascending=False)
#patterns2=pyfpgrowth.find_frequent_patterns(d2, 163)
#
#69 patterns as opposed to 76 from pyspark